In [1]:
# CONFIG
ICL_model_name = "tiiuae/falcon-7b" # note: "meta-llama/Meta-Llama-3-8B", "EleutherAI/pythia-6.9b", "tiiuae/falcon-7b", "meta-llama/Meta-Llama-3-70B", "tiiuae/falcon-40b"
quantized = False if ICL_model_name in ["meta-llama/Meta-Llama-3-8B", "EleutherAI/pythia-6.9b", "tiiuae/falcon-7b"] else True
token = "your token here"
ICL_selected_token_type = "label_words" # "none" (natural ICL sample end), "label_words", "last_sentence_token"
Hidden_calibration_selected_token_type = "label_words"
k = 4 # 0, 1, 2, 4, 8, 12
dataset_index = 1 # 0, 1, 2, 3, 4, 5
corr_label = True

In [2]:
import sys
sys.path.append("..")
from util import load_model_and_data, kernel_alignment, inference
from StaICC_Expe import StaICC

bench = StaICC.Normal(k)
prompts, queries = load_model_and_data.load_data_from_StaICC_experimentor(bench[dataset_index], ICL_selected_token_type, corr_label)

Loading data...

1 in 10 Data loaded:  GLUE-SST2 

2 in 10 Data loaded:  rotten_tomatoes 

3 in 10 Data loaded:  financial_phrasebank 

4 in 10 Data loaded:  SST5 

5 in 10 Data loaded:  TREC 

6 in 10 Data loaded:  AGNews 

7 in 10 Data loaded:  Subjective 

8 in 10 Data loaded:  tweet_eval_emotion 

9 in 10 Data loaded:  tweet_eval_hate 

10 in 10 Data loaded:  hate_speech_18 

Data loaded successfully.

Initializing experimentor on k = 4...

Ready.



In [3]:
vars_dict = vars() if "ICL_model" in vars() else locals()
if not "ICL_model" in vars_dict:
    ICL_model, ICL_tknz = load_model_and_data.load_ICL_model(ICL_model_name, huggingface_token = token, quantized = quantized)
    loaded = True

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
ICL_hidden_states = inference.ICL_inference_to_hidden_states(ICL_model, ICL_tknz, prompts)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 512/512 [01:37<00:00,  5.26it/s]


In [5]:
from util import layered_hidden_calibration
import functools

inferencer = layered_hidden_calibration.layered_hidden_calibration(
    bench[dataset_index].get_label_space(), 
    len(ICL_hidden_states), 
    Hidden_calibration_selected_token_type, 
    corr_label if Hidden_calibration_selected_token_type == "label_words" else True
)
inferencer.train(
    bench[dataset_index].prompt_former, 
    functools.partial(inference.ICL_inference_to_hidden_states_transposed, model = ICL_model, tokenizer = ICL_tknz),
    calibration_set = bench[dataset_index].calibration_set(),
    calibration_number = 256,
    k = k
)

Process: 0%, 1 in 256 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.96it/s]


Process: 0%, 2 in 256 

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  4.50it/s]


Process: 1%, 3 in 256 

...

Process: 99%, 255 in 256 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.29it/s]


Process: 100%, 256 in 256 >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  5.43it/s]


Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibration Training Finished.

Calibrat

In [6]:
inf_res = inferencer.batched_layered_inference(ICL_hidden_states)

In [7]:
res = []
for i in range(len(ICL_hidden_states)):
    res.append(bench[dataset_index](forward_inference = None, input_prediction = inf_res[i] + inf_res[i])[0]['accuracy'])

In [8]:
res

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.99609375]

In [9]:
# Result file organization:

'''
dataset index:

1 in 10 Data loaded:  GLUE-SST2 

2 in 10 Data loaded:  rotten_tomatoes 

3 in 10 Data loaded:  financial_phrasebank 

4 in 10 Data loaded:  SST5 

5 in 10 Data loaded:  TREC 

6 in 10 Data loaded:  AGNews 

7 in 10 Data loaded:  Subjective 

8 in 10 Data loaded:  tweet_eval_emotion 

9 in 10 Data loaded:  tweet_eval_hate 

10 in 10 Data loaded:  hate_speech_18 
'''

import pickle

data_file_name = "data/" + ICL_model_name.replace('/', '_')+ "," + ICL_selected_token_type + ",HiddenC," + Hidden_calibration_selected_token_type  + ',' + ICL_selected_token_type + ',' + ("corr," if corr_label else "wrong,") + str(k) + ',' + str(dataset_index + 1) + ".pickle"
with open(data_file_name, 'wb') as f:
    pickle.dump(res, f)